In [2]:
# fit a mask rcnn on the kangaroo dataset
from os import listdir
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import json
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

In [3]:
class LoadDataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir):
        self.add_class("dataset",1, "built_up_area")
        self.add_class("dataset",2,"water_body")
        self.add_class("dataset",3,"vegetation")
        self.add_class("dataset",4,"open_land")
        
        
        current_path = dataset_dir
        images_dir = os.path.join(current_path, 'images')
        annotations_dir = os.path.join(current_path, 'annots')
        
        # find all images
        for filename in listdir(images_dir):
            # extract image id
            image_id = filename.split('.')[0]
            image_file = os.path.join(images_dir, filename)    
            annotation_file = os.path.join(annotations_dir, image_id + '.json')
            with open(annotation_file, 'r') as f:            
                data = json.load(f)
            annotations = data['shapes']  # don't need the dict keys
            for a in annotations:
                class_id = [int(self.get_coord(a)[1])]
                polygons = [self.get_coord(a)[0]]
                image = skimage.io.imread(image_file)
                height, width = image.shape[:2]
                # add to dataset
                self.add_image(
                "dataset",  ## for a single class just add the name here
                image_id= image_id,  # use file name as a unique image id
                path=image_file,
                width=width, height=height,
                polygons=polygons,
                class_ids=class_id)
        '''
    input = dictionary
    this function will retrun x,y coordinates of a annotation in folllowing format
    shape_attribute = {
       'x' = [101,102]
       'y' = [112,115]
    }
    '''
    def get_coord(self,json_dict):
        shape_attribute = {}
        map_dict = {'built_up_area': '1',
                    'water_body': '2',
                    'vegetation': '3',
                    'open_land': '4'}
        class_label = json_dict['label']
        class_id = map_dict[class_label]
        x = [point[0] for point in json_dict['points']]
        y = [point[1] for point in json_dict['points']]
        shape_attribute['x'] = x
        shape_attribute['y'] = y
        return shape_attribute,class_id
    
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a balloon dataset image, delegate to parent class.
        image_info = self.image_info[image_id]

        class_ids = image_info['class_ids']
        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['y'], p['x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        #class_ids=np.array([self.class_names.index(shapes[0])])
#         print("info['class_ids']=", info['class_ids'])
        class_ids = np.array(class_ids, dtype=np.int32)
        return mask, class_ids
    
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "damage":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)
    

In [4]:
# define a configuration for the model
class DataConfig(Config):
	# define the name of the configuration
	NAME = "vegetation_cfg"
	# number of classes (background + kangaroo)
	NUM_CLASSES = 1 + 4
	# number of training steps per epoch
	STEPS_PER_EPOCH = 10
    
config = DataConfig()

In [5]:
#preparing trainset
train_set = LoadDataset()
train_set.load_dataset('veg_samp')
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

Train: 25


In [6]:
#example of a train image
print(train_set.image_info[0])
print('*'*100)
print(train_set.image_info[1])
print('*'*100)
print(train_set.image_info[2])

{'height': 960, 'width': 960, 'path': 'veg_samp/images/000000000.jpg', 'source': 'dataset', 'class_ids': [1], 'polygons': [{'x': [78.66666666666666, 70.66666666666666, 17.333333333333343, 24.0, 225.33333333333331, 224.0, 380.0, 384.0, 496.0, 494.66666666666663, 858.6666666666666, 789.3333333333334, 780.0, 697.3333333333334, 704.0, 697.3333333333334, 816.0, 756.0, 686.6666666666666], 'y': [21.333333333333332, 286.6666666666667, 306.6666666666667, 445.3333333333333, 448.0, 585.3333333333334, 581.3333333333334, 654.6666666666666, 665.3333333333334, 944.0, 952.0, 918.6666666666666, 740.0, 726.6666666666666, 621.3333333333334, 484.0, 460.0, 240.0, 8.0]}], 'id': '000000000'}
****************************************************************************************************
{'height': 960, 'width': 960, 'path': 'veg_samp/images/000000000.jpg', 'source': 'dataset', 'class_ids': [2], 'polygons': [{'x': [705.3333333333334, 710.6666666666666, 852.0, 845.3333333333334], 'y': [509.3333333333333, 6

In [7]:
# prepare config
config = DataConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                17
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0, 'rpn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [ ]:
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
# train weights (output layers or 'heads')
model.train(train_set, train_set, learning_rate=config.LEARNING_RATE, epochs=2, layers='heads')







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead

Starting at epoch 0. LR=0.001

Checkpoint Path: ./vegetation_cfg20191030T1238/mask_rcnn_vegetation_cfg_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_m

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple worker



Epoch 1/2
 2/10 [=====>........................] - ETA: 10:17 - loss: 5.9140 - rpn_class_loss: 0.0394 - rpn_bbox_loss: 0.3332 - mrcnn_class_loss: 2.9406 - mrcnn_bbox_loss: 1.0428 - mrcnn_mask_loss: 1.5580

In [ ]:
import gc
gc.collect()